### Modelos baseados em árvores

<br>

Ao longo do módulo, discutimos bastante as árvores de decisão, bem como ensemble de árvores, como Random Forest e algoritmos do tipo boosting.

Esses __ensembles acabam tendo muitos hiperparâmetros;__ escolhe-los de forma manual acaba sendo muito custoso e tedioso. 

Neste exercício, vamos discutir a respeito da metolodia __grid-search__, que otimiza essa busca de hiperparâmetros.

Considere o dataset abaixo (basta executar as células):

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [ ]:
#problema de regressão

X, y = load_diabetes().data, load_diabetes().target
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(331, 10) (111, 10) (331,) (111,)


Imagine que queremos testar - usando cross-validation - várias instâncias de Random Forests: com 10 árvores, com 100 árvores, com 1000 árvores, com profundidade máxima 1, 5, 10. 

Como podemos proceder? O código abaixo exemplifica um jeito:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

for n_est in [10,100,100]:
    for prof in [1,5,10]:
        rf = RandomForestRegressor(n_estimators=n_est, max_depth=prof)
        cvres = cross_val_score(estimator=rf, X = Xtrain, y = ytrain, cv = 3, scoring='r2')
        print("estimators: ", n_est, " prof: ", prof, " | R2 mean / std: ", cvres.mean(), ' / ', cvres.std())

estimators:  10  prof:  1  | R2 mean / std:  0.3169445041685421  /  0.03576810884643167
estimators:  10  prof:  5  | R2 mean / std:  0.39063311361896097  /  0.03857920940036989
estimators:  10  prof:  10  | R2 mean / std:  0.3577664804450773  /  0.062245028024447806
estimators:  100  prof:  1  | R2 mean / std:  0.3358780366869931  /  0.03497061550809594
estimators:  100  prof:  5  | R2 mean / std:  0.42263410120621253  /  0.03769116664926871
estimators:  100  prof:  10  | R2 mean / std:  0.4132914390651277  /  0.04079830873970563
estimators:  100  prof:  1  | R2 mean / std:  0.3351891182366678  /  0.03962897694542215
estimators:  100  prof:  5  | R2 mean / std:  0.42249018882489003  /  0.0408945678541201
estimators:  100  prof:  10  | R2 mean / std:  0.4154343348101334  /  0.04723485912034823


Podemos, com algum trabalho, escolher o melhor modelo.

Se quisermos testar mais parâmetros, podemos aumentar nosso loop... mais isso vai ficando cada vez mais complicado.

A proposta do __grid-search__ é justamente fazer isso de forma mais automática!

Podemos importar a função GridSearchCV do módulo model_selection do sklearn e usá-la para isso. 
Na prática, precisamos definir um __estimador base__ para o grid. Além disso, precisamos definir um __dicionário de parâmetros__ a ser testado. Ainda, definiremos a quantidade de folds para cross-validation e qual a métrica de performance que queremos otimizar:

### GRID-SEARCH

In [ ]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [27]:
#definindo o estimador base
estimador_base = RandomForestRegressor()

#definindo o dicionario de parâmetros do modelo
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [ ]:
grid = GridSearchCV(estimator = estimador_base, 
                    param_grid = params_RF, 
                    scoring = 'r2', 
                    cv = 3)

grid

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

In [ ]:
#treinando os modelos no grid
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 10], 'n_estimators': [10, 1000]},
             scoring='r2')

O objeto "grid", após o treinamento acima, conterá várias informações muito relevantes. 

__1- "best_params_":__ retorna os melhores parâmetros, de acordo com a métrica de performance avaliada na cross-validation;

__1- "best_score_":__ retorna o melhor score - métrica de performance - nos dados de validação;

__1- "best_estimator_":__ retorna o melhor modelo, já treinado;

__1- "cv_results_":__ retorna uma visão geral dos resultados.

In [ ]:
grid.best_params_

{'max_depth': 10, 'n_estimators': 1000}

In [ ]:
grid.best_score_

0.4251447728141627

In [ ]:
grid.best_estimator_

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [ ]:
grid.cv_results_

{'mean_fit_time': array([0.02224716, 1.43986114, 0.01980631, 2.06266101]),
 'mean_score_time': array([0.00257349, 0.09718347, 0.00174737, 0.09713626]),
 'mean_test_score': array([0.39983753, 0.40781437, 0.36359401, 0.42514477]),
 'param_max_depth': masked_array(data=[2, 2, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[10, 1000, 10, 1000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'n_estimators': 10},
  {'max_depth': 2, 'n_estimators': 1000},
  {'max_depth': 10, 'n_estimators': 10},
  {'max_depth': 10, 'n_estimators': 1000}],
 'rank_test_score': array([3, 2, 4, 1], dtype=int32),
 'split0_test_score': array([0.32596407, 0.35044552, 0.30137022, 0.37555185]),
 'split1_test_score': array([0.45163003, 0.44571691, 0.40931155, 0.45631865]),
 'split2_test_score': array([0.42191849, 0.42728069, 0.38010026

### EXERCICIOS

In [4]:
#importando a função
from sklearn.model_selection import GridSearchCV

In [5]:
from sklearn.ensemble import  GradientBoostingRegressor

In [6]:
from sklearn.neighbors import KNeighborsRegressor

In [7]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
from sklearn.model_selection import train_test_split

__Exercício 1:__ Utilizando o dataset abaixo, faça um grid_search com KNN's, Random Forests e GradientBoostings e retorne o melhor modelo de cada tipo.

__Obs.:__ Lembre-se de fazer um pré-processamento nos dados!

__Obs.: Exercicio resolvido no Google Colab__

In [9]:
#preco_mediano_das_casas é a variável target
df = pd.read_csv("/content/drive/MyDrive/CIENTISTA DE DADOS/MENTORAMA - AULAS/MODULO 12/preco_casas.csv")
print(df.shape)
df.head()

(20640, 10)


,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano,preco_mediano_das_casas,proximidade_ao_mar
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,PERTO DA BAÍA
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,PERTO DA BAÍA
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,PERTO DA BAÍA
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,PERTO DA BAÍA
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,PERTO DA BAÍA


In [10]:
#verificando valores nulos
df.isnull().sum()

longitude                    0
latitude                     0
idade_mediana_das_casas      0
total_comodos                0
total_quartos              207
populacao                    0
familias                     0
salario_mediano              0
preco_mediano_das_casas      0
proximidade_ao_mar           0
dtype: int64

In [11]:
#Substituindo os valores faltantes pela Mediana
mediana = df['total_quartos'].median()
df['total_quartos'].fillna(mediana, inplace=True)

In [12]:
#Variaveis Categoricas
df['proximidade_ao_mar'].unique()

array(['PERTO DA BAÍA', '<1H OCEANO', 'INTERIOR', 'PERTO OCEANO', 'ILHA'],
      dtype=object)

In [13]:
#Definindo as Variaveis de Estudo
X = df.drop(['preco_mediano_das_casas'], axis=1)
y = df.preco_mediano_das_casas.values.reshape(-1,1)

In [14]:
#Removendo a Variavel Categorica
X.drop(columns ='proximidade_ao_mar', inplace=True)

In [15]:
X

,longitude,latitude,idade_mediana_das_casas,total_comodos,total_quartos,populacao,familias,salario_mediano
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462
...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672


In [16]:
#Aplicando o One Hot Encoder
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()

df_prox_mar_OHE = cat_encoder.fit_transform(df[['proximidade_ao_mar']]).toarray()

X = np.c_[X,df_prox_mar_OHE]

In [17]:
X.shape

(20640, 13)

In [18]:
#Dataset de Treino, e Teste
x1_treino, x1_teste, y1_treino, y1_teste = train_test_split(X, y, test_size = 0.2)

In [19]:
#definindo os estimadores

knn = KNeighborsRegressor()
rf = RandomForestRegressor()
gbr =  GradientBoostingRegressor( )

#definindo o dicionario de parâmetros dos modelos
params_knn = {"n_neighbors":[1,2,3,4,5,6]}
params_rf = {"n_estimators":[5, 10, 100], "max_depth":[1,2,5,6]}
params_gbr = {"learning_rate":[0.5, 0.6, 0.7], "n_estimators":[2, 10 , 20, 50]}

In [20]:
grid_knn = GridSearchCV(estimator = knn, 
                    param_grid = params_knn, 
                    scoring = 'r2', 
                    cv = 3)

grid_rf = GridSearchCV(estimator = rf, 
                    param_grid = params_rf, 
                    scoring = 'r2', 
                    cv = 3)

grid_gbr = GridSearchCV(estimator = gbr, 
                    param_grid = params_gbr, 
                    scoring = 'r2', 
                    cv = 3)


In [21]:
#treinando os modelos no grid

grid_knn.fit(x1_treino, y1_treino)
grid_rf.fit(x1_treino, y1_treino)
grid_gbr.fit(x1_treino, y1_treino)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.5, 0.6, 0.7],
                         'n_estimators': [2, 10, 20, 50]},
             scoring='r2')

In [22]:
print("MELHORES RESULTADOS DOS MODELOS: ")
print("====="*10)
print("KNN: ")
print("N° De vizinhos: {} || R2: {:.3f}".format(grid_knn.best_params_['n_neighbors'],grid_knn.best_score_))
print("====="*10)
print("RANDOM FOREST REGRESSOR: ")
print("N° Estimadores: {} || Max Depth: {} || R2: {:.3f}".format(grid_rf.best_params_['n_estimators'], 
                                                                                             grid_rf.best_params_['max_depth'], 
                                                                                             grid_rf.best_score_))
print("====="*10)
print("GRADIENT BOOSTING REGRESSOR: ")
print("Learning Rate: {} || N° Estimadores: {} || R2: {:.3f}".format(grid_gbr.best_params_["learning_rate"], 
                                                                                             grid_gbr.best_params_['n_estimators'], 
                                                                                             grid_gbr.best_score_))

MELHORES RESULTADOS DOS MODELOS: 
KNN: 
N° De vizinhos: 6 || R2: 0.273
RANDOM FOREST REGRESSOR: 
N° Estimadores: 100 || Max Depth: 6 || R2: 0.686
GRADIENT BOOSTING REGRESSOR: 
Learning Rate: 0.5 || N° Estimadores: 50 || R2: 0.790


__Exercício 2:__ Crie uma classe para comparar o grid_search dentre vários modelos distintos.
    
    
Essa classe, gridSearchAll(), já está pré-desenvolvida no código abaixo. O exercício consiste de __completar essa classe.__ Para isso, crie o métodos fit_all, que irá treinar, usando grid_search, todos os grids que tenham sido pré-construídos e inseridos na classe.
Ainda, a quantidade de folds para a validação cruzada no grid_search deve ser implementada no método construtor da classe, bem como qual a métrica de performance a ser avaliada. 
Finalmente, salve o melhor modelo de cada grid e tenha um método best_all_grid_models que retorna o melhor modelo dentre todos os grids.

In [23]:
class gridSearchAll():
    
    def __init__(self, scoring, num_folds):
        self.grid_models = []
        self.scoring = scoring
        self.num_folds = num_folds
        self.best = []
    
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])
    def fit_all(self, X, y):
      for est, param in self.grid_models:
        grid = GridSearchCV(estimator = est, 
                    param_grid = param, 
                    scoring = self.scoring, 
                    cv = self.num_folds)
        grid.fit(X, y)
        self.best.append(grid)

    def best_all_grid_models(self):
      best_models = []
      for grid in self.best:
        best_models.append(grid.best_estimator_)
      print("Melhores Modelos:")
      print(best_models)

In [24]:
gd = gridSearchAll('r2',3)

In [25]:
gd.grid_models

[]

In [28]:
params_RF = {"n_estimators":[10,1000], "max_depth":[2,10]}

In [29]:
params_RF

{'max_depth': [2, 10], 'n_estimators': [10, 1000]}

In [30]:
gd.insert_model(estimator_base = RandomForestRegressor(), param_grid = params_RF)

In [31]:
gd.grid_models

[[RandomForestRegressor(), {'max_depth': [2, 10], 'n_estimators': [10, 1000]}]]

In [32]:
gd.insert_model(estimator_base = KNeighborsRegressor(), param_grid = {"n_neighbors":[1,2,10]})

In [33]:
gd.grid_models

[[RandomForestRegressor(), {'max_depth': [2, 10], 'n_estimators': [10, 1000]}],
 [KNeighborsRegressor(), {'n_neighbors': [1, 2, 10]}]]

In [34]:
gd.fit_all(x1_treino, y1_treino)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 

In [35]:
resultado = gd.best_all_grid_models()
resultado

Melhores Modelos:
[RandomForestRegressor(max_depth=10, n_estimators=1000), KNeighborsRegressor(n_neighbors=10)]


__Exercício 3:__ Usando a classe criada, analise novamente os modelos criados no exercício 1.

knn = KNeighborsRegressor()<br>
rf = RandomForestRegressor( )<br>
gbr =  GradientBoostingRegressor( )<br>

params_knn = {"n_neighbors":[1,2,3,4,5,6]}<br>
params_rf = {"n_estimators":[5, 10, 100], "max_depth":[1,2,5,6]}<br>
params_gbr = {"learning_rate":[0.5, 0.6, 0.7], "n_estimators":[2, 10, 20, 50]} <br>


In [36]:
exe = gridSearchAll('r2',5)

In [37]:
exe.insert_model(KNeighborsRegressor(), params_knn)
exe.insert_model(RandomForestRegressor(), params_rf)
exe.insert_model(GradientBoostingRegressor(), params_gbr)

In [39]:
exe.grid_models

[[KNeighborsRegressor(), {'n_neighbors': [1, 2, 3, 4, 5, 6]}],
 [RandomForestRegressor(),
  {'max_depth': [1, 2, 5, 6], 'n_estimators': [5, 10, 100]}],
 [GradientBoostingRegressor(),
  {'learning_rate': [0.5, 0.6, 0.7], 'n_estimators': [2, 10, 20, 50]}]]

In [40]:
exe.fit_all(x1_treino, y1_treino)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 

In [41]:
resultado2 = exe.best_all_grid_models()
resultado2

Melhores Modelos:
[KNeighborsRegressor(n_neighbors=6), RandomForestRegressor(max_depth=6), GradientBoostingRegressor(learning_rate=0.7, n_estimators=50)]


## Resultado do EXE1: <br>
KNN: 
N° De vizinhos: 6

RANDOM FOREST REGRESSOR: 
N° Estimadores: 100 || Max Depth: 6 

GRADIENT BOOSTING REGRESSOR: 
Learning Rate: 0.5 || N° Estimadores: 50 
<br>
<br>
OBS.: O resultado do KNN e Random Forest do EXE3 com o EXE1 foram iguais, porem o Learning Rate do Gadient Boosting Regressor do EXE3 foi de 0.7 e do EXE1 0.5, mas o n° de estimadores foi igual